In [37]:
from func import *

import glob 
import json
import subprocess
import getpass
import os

In [77]:
volume_txt_file = 'datasets/nako/volumes.txt'
data_dir = "/home/abhijit/nas_drive/Data_WholeBody/NAKO/NAKO_200/MRI"
label_dir = 'datasets/lablmaps/NAKO'

n4_corrected_data_dir = "temp/NAKO/n4_corrected"
stictched_data_dir = "/home/abhijit/nas_drive/Data_WholeBody/NAKO/NAKO_200/stitched"
stitched_n4_corrected_data_dir = "/home/abhijit/nas_drive/Data_WholeBody/NAKO/NAKO_200/stitched_n4_corrected"
# processed_dir = '/home/abhijit/nas_drive/Data_WholeBody/NAKO/NAKO_200/processed'


processed_path = 'temp/NAKO/'
processed_dir = 'temp/NAKO/processed'

In [9]:
def load_nako_file_paths(load_from_txt_file=True):
    volumes_to_use = []
    if load_from_txt_file:
        with open(volume_txt_file) as file_handle:
                volumes_to_use = file_handle.read().splitlines()
    else:
        volumes_to_use = [name for name in os.listdir(data_dir)]
    
    file_paths = {}
    
    for vol in volumes_to_use:
        
        opp_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_opp/**.nii.gz')
        in_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_in/**.nii.gz')
        f_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_F/**.nii.gz')
        w_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_W/**.nii.gz')
        
        labelmap_paths = glob.glob(f'{label_dir}/{vol}/**')
        
        vol_madals_paths = dict(
        OPP=opp_paths,
        IN=in_paths,
        F=f_paths,
        W=w_paths
        )
        file_paths[str(vol)]=dict(
            volume_paths=vol_madals_paths,
            label_paths=labelmap_paths,
        )
    return file_paths
        

In [10]:
file_paths = load_nako_file_paths()
# print(json.dumps(file_paths,sort_keys=True, indent=4))

In [59]:
def rescale(in_image, vol_id, original_filename):
    new_filename = original_filename.split('/')[-1].split('.')[0]
    in_image_data = in_image.get_fdata()
    o_min = np.min(in_image_data)
    o_max = np.max(in_image_data)
    
    if o_min<0:
        print('neagtive value detected')

    in_image_data_scaled = in_image_data + 10
    
    u_min = np.min(in_image_data_scaled)
    u_max = np.max(in_image_data_scaled)
    
    in_image_scaled = nb.Nifti1Image(in_image_data_scaled, in_image.affine.copy(), in_image.header.copy())
    create_if_not(f'{n4_corrected_data_dir}/vol/{vol_id}')
    save_volume(in_image_scaled, f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled')

#     password = getpass.getpass()
#     p = subprocess.Popen(["sudo", "sh", "./n4-anne.sh"], stdout=subprocess.STDOUT, stderr=subprocess.PIPE)
#     stdout, stderr = p.communicate('{}\n'.format(password)) # '{}\n'.format('Abhi12345$$')
#     if p.returncode != 0:
#         print(stderr) 
#     else:
#         print(stdout)
#     command = "sudo sh ./n4-anne.sh" #can be any command but don't forget -S as it enables input from stdin
#     os.system('echo %s | %s' % (password, command))

    return dict(
        in_n4_scaled_path= f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled.nii.gz',
        in_n4_bias_field= f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled_bias_field.nii.gz',
        in_n4_corrected= f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled_corrected.nii.gz',
        MIN= u_min,
        MAX= u_max
    )

def apply_bias_field(opp_img, bias_field_img, opp_file, n4_dict, vol_id):
    new_filename = opp_file.split('/')[-1].split('.')[0]
    
    opp_img_data = opp_img.get_fdata()
    bias_field_img_data = bias_field_img.get_fdata()
    corrected_img_data = opp_img_data / bias_field_img_data
    
    orig_min = n4_dict['MIN']
    orig_max = n4_dict['MAX']
    current_min = np.min(corrected_img_data)
    current_max = np.max(corrected_img_data)
    downscaled_img_data = (corrected_img_data - current_min) * (orig_max - orig_min)/(current_max - current_min) + orig_min

    
    downscaled_img = nb.Nifti1Image(downscaled_img_data, opp_img.affine.copy(), opp_img.header.copy())
    create_if_not(f'{n4_corrected_data_dir}/vol/{vol_id}')

    save_volume(downscaled_img, f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_corrected')

    print(n4_dict)
    n4_dict['opp_n4_corrected'] = f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_corrected.nii.gz'
    return n4_dict
    

In [60]:
# Individual N4 Corrections
n4_dict = {}
for vol in file_paths.keys():
    print(f'n4 processing part-1 started with {vol}...')
    n4_dict[vol] = []
    vol_parts = [[file, read_ras(file)] for file in file_paths[vol]['volume_paths']['IN']]
    for orig_file, in_image in vol_parts:
        n4_dict[vol].append(rescale(in_image, vol, orig_file))
        
# EXECUTE 'sudo sh ./n4-anne.sh' from command line for n4 bais field generation of all in imgs.

for vol in file_paths.keys():
    print(f'applying n4 bias field on opp scans of {vol}...')
    vol_parts = [[read_ras(opp_file), read_ras(n4_counterpart['in_n4_bias_field']), opp_file] for opp_file, n4_counterpart in zip(file_paths[vol]['volume_paths']['OPP'],n4_dict[vol])]
    idx = 0
    file_paths[vol]['n4_paths'] = []
    for opp_img, bias_field_img, opp_file in vol_parts:
        file_paths[vol]['n4_paths'].append(apply_bias_field(opp_img, bias_field_img, opp_file, n4_dict[vol][idx], vol))
        idx+=1
#         n4_dict[vol].append(rescale(opp_image, vol, n4_dict[vol]))

n4 processing part-1 started with 100008...
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625       0.            0.         -231.09375   ]
 [   0.            1.40625       0.         -214.43591309]
 [   0.            0.            3.         -384.76696777]
 [   0.            0.            0.            1.        ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625       0.            0.         -231.09375   ]
 [   0.            1.40625       0.         -214.43591309]
 [   0.            0.            3.         -576.76696777]
 [   0.            0.            0.            1.        ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625       0.            0.         -231.09375   ]
 [   0.            1.40625       0.         -214.43591309]
 [   0.            0.            3.         -804.76696777]
 [   0.            0.         

Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -223.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -862.5    ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 88)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -223.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -862.5    ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 88)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[ 1.4062500e+00  0.0000000e+00  0.0000000e+00 -2.2359375e+02]
 [ 0.0000000e+00  1.4062500e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.1025000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (320, 260, 88)
Transforming Images to RAS.....
R

In [69]:
for vol in file_paths.keys():
    print(f'started with {vol}...')
    create_if_not(f'{processed_dir}/vol/{vol}')
    file_paths[vol]['vol_stitched_path'] = {}
    for modality_key in file_paths[vol]['volume_paths'].keys():
        print(f"processing {modality_key}")
        orig_modal_key = modality_key
        
        if modality_key == 'OPP':
            vol_parts = [read_ras(data_dict['opp_n4_corrected']) for data_dict in file_paths[vol]['n4_paths']]
            modality_key = modality_key+'_n4_corrected'
        else:
            vol_parts = [read_ras(file) for file in file_paths[vol]['volume_paths'][modality_key]]
        ras_stitched = multi_vol_stitching(vol_parts)
        save_volume(ras_stitched, f'{processed_dir}/vol/{vol}/{modality_key}_ras_stitched')
        file_paths[vol]['vol_stitched_path'][f'{orig_modal_key}'] = f'{processed_dir}/vol/{vol}/{modality_key}_ras_stitched.nii.gz'
    
    
    
    

started with 100008...
processing OPP
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625       0.            0.         -231.09375   ]
 [   0.            1.40625       0.         -214.43591309]
 [   0.            0.            3.         -384.76696777]
 [   0.            0.            0.            1.        ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625       0.            0.         -231.09375   ]
 [   0.            1.40625       0.         -214.43591309]
 [   0.            0.            3.         -576.76696777]
 [   0.            0.            0.            1.        ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625       0.            0.         -231.09375   ]
 [   0.            1.40625       0.         -214.43591309]
 [   0.            0.            3.         -804.76696777]
 [   0.            0.            0. 

Affine:[[   1.40625    0.         0.      -223.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -634.5    ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -223.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -862.5    ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 88)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[ 1.4062500e+00  0.0000000e+00  0.0000000e+00 -2.2359375e+02]
 [ 0.0000000e+00  1.4062500e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.1025000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (320, 260, 88)
Transforming Images to RAS.....


In [74]:
n4_dict = {}
for vol in file_paths.keys():
    print(f'n4 processing part-2 started with {vol}...')
    n4_dict[vol] = {}
    in_stitched_file_path, in_stitched_img = file_paths[vol]['vol_stitched_path']['IN'], read_ras(file_paths[vol]['vol_stitched_path']['IN'])
    n4_dict[vol]['stitched_rescaled_data'] = rescale(in_stitched_img, vol, in_stitched_file_path)
        
# EXECUTE 'sudo sh ./n4-anne.sh' from command line for n4 bais field generation of all in imgs.



n4 processing part-2 started with 100008...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -2.31093750e+02]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -2.14435913e+02]
 [ 0.00000000e+00  0.00000000e+00  3.00000000e+00 -1.04476697e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
neagtive value detected
n4 processing part-2 started with 100020...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.2359375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.1025000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
neagtive value detected


In [79]:
for vol in file_paths.keys():
    print(f'applying n4 bias field on stitched opp scans of {vol}...')

    opp_file = file_paths[vol]['OPP_vol_stitched_path']
    n4_counterpart = n4_dict[vol]['stitched_rescaled_data']
    opp_img = read_ras(opp_file)
    bias_field_img = read_ras(n4_counterpart['in_n4_bias_field'])

    file_paths[vol]['n4_paths_2'] = apply_bias_field(opp_img, bias_field_img, opp_file, n4_dict[vol]['stitched_rescaled_data'], vol)

applying n4 bias field on stitched opp scans of 100008...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -2.31093750e+02]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -2.14435913e+02]
 [ 0.00000000e+00  0.00000000e+00  3.00000000e+00 -1.04476697e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -2.31093750e+02]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -2.14435913e+02]
 [ 0.00000000e+00  0.00000000e+00  3.00000000e+00 -1.04476697e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
{'in_n4_scaled_path': 'temp/NAKO/n4_corrected/vol/100008/IN_ras_stitched_n4_scaled.nii.gz', 'in_n4_bias_field': 'temp/NAKO/n4_corrected/vol/100008/IN_ras_stitched_n4_scaled_bias_

In [80]:
print(json.dumps(file_paths,sort_keys=True, indent=4))

{
    "100008": {
        "F_vol_stitched_path": "temp/NAKO/processed/vol/100008/F_ras_stitched.nii.gz",
        "IN_vol_stitched_path": "temp/NAKO/processed/vol/100008/IN_ras_stitched.nii.gz",
        "OPP_vol_stitched_path": "temp/NAKO/processed/vol/100008/OPP_n4_corrected_ras_stitched.nii.gz",
        "W_vol_stitched_path": "temp/NAKO/processed/vol/100008/W_ras_stitched.nii.gz",
        "label_paths": [
            "datasets/lablmaps/NAKO/100008/100008_3D_GRE_TRA_opp_3D_GRE_TRA_2_Adrenal gland (left).nrrd",
            "datasets/lablmaps/NAKO/100008/100008_3D_GRE_TRA_opp_3D_GRE_TRA_2-2.nrrd_Spleen-5.nrrd",
            "datasets/lablmaps/NAKO/100008/100008_3D_GRE_TRA_opp_3D_GRE_TRA_2_Gallbladder (1).nrrd",
            "datasets/lablmaps/NAKO/100008/100008_3D_GRE_TRA_opp_3D_GRE_TRA_2_Adrenal gland (right).nrrd",
            "datasets/lablmaps/NAKO/100008/10008_3D_GRE_TRA_opp_3D_GRE_TRA_2.nrrd_subcutan fat.nrrd",
            "datasets/lablmaps/NAKO/100008/100008_3D_GRE_TRA_opp_3D_GRE_T